<a href="https://colab.research.google.com/github/wkowalc/POC_VET_EYE_AI/blob/main/POC_VET_EYE_AI_%2B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**System predykcyjny płodności loch trzody chlewnej**

Źródło danych: https://data.mendeley.com/datasets/z6p55dvbxj/1

**Pobranie bibliotek**

In [ ]:
# Instalacja TensorFlow
!pip install tensorflow matplotlib --quiet

# Importowanie bibliotek
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers, regularizers
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Sprawdzenie wersji TensorFlow
print(f"TensorFlow version: {tf.__version__}")

TensorFlow version: 2.18.0


**Importowanie danych z MyDrive**

Stworzone zostały 5 Klas o rożnej ilości danych wejściowych:

1.   PecherzykDuzy (310)
2.   PecherzykMaly (180)
3.   PecherzykSredni (278)
4.   PecherzykPrzedowulacyjny (100)
5.   Random (115)

In [ ]:
import cv2
import numpy as np
import os
from google.colab import drive

# Google Drive
drive.mount('/content/drive')

# Foldery z danymi
data_path = "/content/drive/MyDrive/TrzodyChlewne"
output_path = "/content/drive/MyDrive/TrzodyChlewne_cleaned2"

# Tworzymy folder na oczyszczone obrazy (żeby nie nadpisać oryginalnych)
os.makedirs(output_path, exist_ok=True)

def remove_yellow_arrows(image_path, save_path):
    # Wczytaj obraz
    image = cv2.imread(image_path)
    if image is None:
        print(f"Błąd: Nie udało się wczytać obrazu: {image_path}")
        return

    # Konwersja do HSV (łatwiejsze wykrywanie żółtego koloru)
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Zakresy dla koloru żółtego (można dostosować)
    lower_yellow = np.array([15, 50, 50])
    upper_yellow = np.array([45, 255, 255])

    # Maska dla żółtych obszarów
    mask = cv2.inRange(hsv, lower_yellow, upper_yellow)

    # Sprawdzenie, czy maska coś wykryła (czy w ogóle są strzałki)
    if np.count_nonzero(mask) == 0:
        print(f"Brak strzałek: {image_path}")
        cv2.imwrite(save_path, image)  # Kopiujemy obraz bez zmian
        return

    # Znalezienie konturów żółtych obiektów
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Sprawdzamy wielkość konturów – usuwamy tylko większe (żeby nie usuwać małych żółtych elementów)
    min_contour_area = 100  # Jeśli strzałki są większe, to wartość podnieść
    for cnt in contours:
        if cv2.contourArea(cnt) > min_contour_area:
            cv2.drawContours(mask, [cnt], -1, 255, thickness=cv2.FILLED)

    # Zamiana wykrytych strzałek na otoczenie
    cleaned_image = cv2.inpaint(image, mask, inpaintRadius=5, flags=cv2.INPAINT_TELEA)

    # Zapisujemy oczyszczony obraz
    cv2.imwrite(save_path, cleaned_image)
    print(f"Oczyszczono: {image_path}")

# Pobranie listy obrazów
for label in os.listdir(data_path):
    label_path = os.path.join(data_path, label)
    output_label_path = os.path.join(output_path, label)

    if os.path.isdir(label_path):
        os.makedirs(output_label_path, exist_ok=True)

        for file in os.listdir(label_path):
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                input_file = os.path.join(label_path, file)
                output_file = os.path.join(output_label_path, file)

                # Usuwamy strzałki, jeśli są
                remove_yellow_arrows(input_file, output_file)

print("Oczyszczone dane znajdują się w folderze:", output_path)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—5.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—28.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/gilt—49.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/gilt—53.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—29.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—32.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—1.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—14.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—31.jpg
Oczyszczono: /content/drive/MyDrive/TrzodyChlewne/PecherzykPrzedowulacyjny/sow—38.jpg
Oczyszczono

**Podzielenie danych na dane treningowe i testowe**

In [ ]:
import os
import shutil
import random

# Folder z danymi
main_folder = '/content/drive/MyDrive/TrzodyChlewne_cleaned2'
train_folder = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_folder = '/content/drive/MyDrive/TrzodyChlewne_test2'

# Stworzenie folderów
os.makedirs(train_folder, exist_ok=True)
os.makedirs(test_folder, exist_ok=True)

# Etykiety
labels = [d for d in os.listdir(main_folder) if os.path.isdir(os.path.join(main_folder, d))]

# Podział danych
for label in labels:
    label_path = os.path.join(main_folder, label)

    # Utworzenie folderów dla treningowych i testowych danych
    label_train_folder = os.path.join(train_folder, label)
    label_test_folder = os.path.join(test_folder, label)
    os.makedirs(label_train_folder, exist_ok=True)
    os.makedirs(label_test_folder, exist_ok=True)

    # Pobranie plików graficznych
    files = [f for f in os.listdir(label_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

    # Losowy podział
    random.shuffle(files)
    split_idx = int(0.8 * len(files))

    train_files = files[:split_idx]
    test_files = files[split_idx:]

    # Przeniesienie plików
    for file in train_files:
        shutil.move(os.path.join(label_path, file), os.path.join(label_train_folder, file))

    for file in test_files:
        shutil.move(os.path.join(label_path, file), os.path.join(label_test_folder, file))

    print(f"Klasa '{label}': {len(train_files)} do treningu, {len(test_files)} do testu.")

print("Dane zostały podzielone!")

Klasa 'PecherzykPrzedowulacyjny': 79 do treningu, 20 do testu.
Klasa 'Random': 112 do treningu, 28 do testu.
Klasa 'PecherzykSredni': 220 do treningu, 56 do testu.
Klasa 'PecherzykMaly': 142 do treningu, 36 do testu.
Klasa 'PecherzykDuzy': 220 do treningu, 55 do testu.
Dane zostały podzielone!


**EKSPERYMENT 1**

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Jeśli używasz generatora

# Ustawienia
image_size = (256, 256)
batch_size = 32
num_classes = 5
train_directory = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_directory = '/content/drive/MyDrive/TrzodyChlewne_test2'

# Budowanie modelu CNN
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

# Kompilacja modelu
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Przygotowanie danych (jeśli używasz ImageDataGenerator)
train_datagen = ImageDataGenerator(rescale=1./255)
train_set = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Definicja `class_weight_dict`, jeśli potrzebujesz wag klas (należy wcześniej obliczyć)
class_weight_dict = None  # Zamień na słownik wag klas, jeśli go używasz

# Trenowanie modelu
model.fit(
    train_set,
    epochs=10,
    class_weight=class_weight_dict  # Jeśli `class_weight_dict` nie jest None
)


Found 964 images belonging to 5 classes.
Epoch 1/10


KeyboardInterrupt: 

**EKSPERYMENT 2**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ustawienia
image_size = (256, 256)
batch_size = 32
num_classes = 5
train_directory = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_directory = '/content/drive/MyDrive/TrzodyChlewne_test2'

# Augmentacja
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Obliczenie wag klas
labels = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels), y=labels)
class_weight_dict = dict(enumerate(class_weights))

# Wczytanie MobileNetV2 (bez klasyfikatora)
base_model = MobileNetV2(weights='imagenet', include_top=False,
                         input_shape=(image_size[0], image_size[1], 3))
base_model.trainable = False  # Zamrażamy bazę dla pierwszej fazy

# Budowanie głowy klasyfikacyjnej
inputs = Input(shape=(image_size[0], image_size[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Kompilacja modelu (pierwsza faza)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacki
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)

# Trening – Faza 1: Zamrożona baza

model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict,
    verbose=1
)

# Faza 2: Odmrożenie MobileNetV2 i dalszy trening

base_model.trainable = True  # Odmrażamy całą bazę

# Kompilacja z niższym learning rate
optimizer_fine = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer_fine,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Dalszy trening (druga faza)
model.fit(
    train_generator,
    epochs=40,  # Możesz dostosować
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr],
    class_weight=class_weight_dict,
    verbose=1
)

Found 964 images belonging to 5 classes.
Found 572 images belonging to 5 classes.


<ipython-input-13-77e78abaf298>:54: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False,


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15


KeyboardInterrupt: 

**EKSPERYMENT 2.1**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ustawienia
image_size = (256, 256)
batch_size = 32
num_classes = 5
train_directory = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_directory = '/content/drive/MyDrive/TrzodyChlewne_test2'


# Augmentacja danych

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Obliczenie wag klas
labels = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels),
                                     y=labels)
class_weight_dict = dict(enumerate(class_weights))


# Budowanie modelu z MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False,
                         input_shape=(image_size[0], image_size[1], 3))
base_model.trainable = False  # Zamrażamy bazę dla pierwszej fazy

inputs = Input(shape=(image_size[0], image_size[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)


# Kompilacja modelu i definicja callbacków

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)


# Trening – Faza 1: Zamrożona baza

model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)


# Faza 2: Odmrożenie MobileNetV2 i dalszy trening

base_model.trainable = True  # Odmrażamy całą bazę

optimizer_fine = tf.keras.optimizers.Adam(learning_rate=5e-5)
model.compile(optimizer=optimizer_fine,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(
    train_generator,
    epochs=40,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)


# Wczytanie najlepszych wag i ewaluacja modelu

model.load_weights('best_model.h5')


Found 964 images belonging to 5 classes.
Found 572 images belonging to 5 classes.


<ipython-input-14-0c4155530b5a>:57: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False,
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

**EKSPERYMENT 2.2**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ustawienia
image_size = (256, 256)
batch_size = 32
num_classes = 5
train_directory = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_directory = '/content/drive/MyDrive/TrzodyChlewne_test2'


# Augmentacja danych

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Obliczenie wag klas
labels = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels),
                                     y=labels)
class_weight_dict = dict(enumerate(class_weights))


# Budowanie modelu z MobileNetV2

base_model = MobileNetV2(weights='imagenet', include_top=False,
                         input_shape=(image_size[0], image_size[1], 3))
# Zamrożenie bazy
base_model.trainable = False

inputs = Input(shape=(image_size[0], image_size[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)


# Kompilacja modelu i definicja callbacków

optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)


# Trening – Faza 1: Zamrożona baza

print("Rozpoczynamy fazę 1: Trening z zamrożoną bazą MobileNetV2")
model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)


# Faza 2: Fine-Tuning – Odmrażamy tylko ostatnie 20 warstw bazy

print("Rozpoczynamy fazę 2: Fine-Tuning")
# Zamrażamy wszystkie warstwy, a następnie odmrażamy tylko ostatnie 20
for layer in base_model.layers:
    layer.trainable = False
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Ustawiamy niższy learning rate dla fine-tuningu
optimizer_fine = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer_fine,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Kontynuujemy trening
model.fit(
    train_generator,
    epochs=40,  # Możesz dostosować liczbę epok
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)


# Wczytanie najlepszych wag i ewaluacja modelu

model.load_weights('best_model.h5')

Found 773 images belonging to 5 classes.
Found 195 images belonging to 5 classes.


<ipython-input-31-71b86b820b59>:58: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(weights='imagenet', include_top=False,


Rozpoczynam fazę 1: Trening z zamrożoną bazą MobileNetV2


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 0.2183 - loss: 2.1195
Epoch 1: val_loss improved from inf to 1.47016, saving model to best_model.h5


...

25/25 ━━━━━━━━━━━━━━━━━━━━ 79s 3s/step - accuracy: 0.4156 - loss: 1.1199 - val_accuracy: 0.4308 - val_loss: 1.1453 - learning_rate: 1.0000e-05
Epoch 28/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4063 - loss: 1.1164
Epoch 28: val_loss did not improve from 1.14530
25/25 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.4066 - loss: 1.1156 - val_accuracy: 0.4205 - val_loss: 1.1465 - learning_rate: 1.0000e-05
Epoch 29/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4460 - loss: 1.0657
Epoch 29: val_loss did not improve from 1.14530
25/25 ━━━━━━━━━━━━━━━━━━━━ 86s 3s/step - accuracy: 0.4457 - loss: 1.0664 - val_accuracy: 0.4205 - val_loss: 1.1457 - learning_rate: 1.0000e-05
Epoch 30/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4554 - loss: 1.0639
Epoch 30: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-06.

Epoch 30: val_loss did not improve from 1.14530
25/25 ━━━━━━━━━━━━━━━━━━━━ 137s 3s/step - accuracy: 0.4551 - loss: 1.0650 - val_accuracy: 0.4205 

25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 3s/step - accuracy: 0.4236 - loss: 1.0691 - val_accuracy: 0.4256 - val_loss: 1.1449 - learning_rate: 2.5000e-06
Epoch 35/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4046 - loss: 1.1417
Epoch 35: val_loss improved from 1.14487 to 1.14467, saving model to best_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 93s 4s/step - accuracy: 0.4053 - loss: 1.1409 - val_accuracy: 0.4256 - val_loss: 1.1447 - learning_rate: 2.5000e-06
Epoch 36/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4412 - loss: 1.0609
Epoch 36: val_loss improved from 1.14467 to 1.14448, saving model to best_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 81s 3s/step - accuracy: 0.4401 - loss: 1.0616 - val_accuracy: 0.4256 - val_loss: 1.1445 - learning_rate: 2.5000e-06
Epoch 37/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4704 - loss: 1.0371
Epoch 37: val_loss improved from 1.14448 to 1.14420, saving model to best_model.h5


25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.4694 - loss: 1.0384 - val_accuracy: 0.4256 - val_loss: 1.1442 - learning_rate: 2.5000e-06
Epoch 38/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4321 - loss: 1.1314
Epoch 38: val_loss did not improve from 1.14420
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 3s/step - accuracy: 0.4325 - loss: 1.1296 - val_accuracy: 0.4256 - val_loss: 1.1445 - learning_rate: 2.5000e-06
Epoch 39/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4481 - loss: 1.0861
Epoch 39: val_loss did not improve from 1.14420
25/25 ━━━━━━━━━━━━━━━━━━━━ 91s 4s/step - accuracy: 0.4482 - loss: 1.0861 - val_accuracy: 0.4308 - val_loss: 1.1445 - learning_rate: 2.5000e-06
Epoch 40/40
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.4389 - loss: 1.1143
Epoch 40: ReduceLROnPlateau reducing learning rate to 1.249999968422344e-06.

Epoch 40: val_loss did not improve from 1.14420
25/25 ━━━━━━━━━━━━━━━━━━━━ 131s 3s/step - accuracy: 0.4387 - loss: 1.1134 - val_accuracy: 0.4308 

**EKSPERYMENT 3**

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, Input
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# Ustawienia
image_size = (300, 300)
batch_size = 32
num_classes = 5
train_directory = '/content/drive/MyDrive/TrzodyChlewne_train2'
test_directory = '/content/drive/MyDrive/TrzodyChlewne_test2'

# Augmentacja danych
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=0.2,
    brightness_range=[0.8, 1.2],
    fill_mode='nearest'
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_directory,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical'
)

# Obliczenie wag klas
labels = train_generator.classes
class_weights = compute_class_weight(class_weight='balanced',
                                     classes=np.unique(labels),
                                     y=labels)
class_weight_dict = dict(enumerate(class_weights))

# Budowanie modelu EfficientNetB3
base_model = EfficientNetB3(weights='imagenet', include_top=False,
                            input_shape=(image_size[0], image_size[1], 3))
base_model.trainable = False  # Zamrażamy bazę na pierwszą fazę treningu

inputs = Input(shape=(image_size[0], image_size[1], 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.4)(x)  # Większy dropout
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.3)(x)
x = layers.Dense(128, activation='relu')(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)
model = models.Model(inputs, outputs)

# Kompilacja modelu
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Callbacki
early_stopping = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)

# Trening – Faza 1: Zamrożona baza
print("Rozpoczynamy fazę 1: Trening z zamrożoną bazą EfficientNetB3")
model.fit(
    train_generator,
    epochs=15,
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)

# Faza 2: Fine-Tuning – Odmrażamy 30 ostatnich warstw
print("Rozpoczynamy fazę 2: Fine-Tuning")
for layer in base_model.layers[:-30]:  # Zamrażamy wszystkie oprócz ostatnich 30
    layer.trainable = False
for layer in base_model.layers[-30:]:
    layer.trainable = True

# Kompilacja z niższym learning rate
optimizer_fine = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer_fine,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Kontynuacja treningu
model.fit(
    train_generator,
    epochs=40,  # Więcej epok dla precyzyjniejszego dostrajania
    validation_data=test_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    class_weight=class_weight_dict,
    verbose=1
)

# Wczytanie najlepszych wag i ewaluacja modelu
model.load_weights('best_model.h5')


Found 964 images belonging to 5 classes.
Found 572 images belonging to 5 classes.
43941136/43941136 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Rozpoczynamy fazę 1: Trening z zamrożoną bazą EfficientNetB3


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


KeyboardInterrupt: 

**Ocena modelu**

In [ ]:
# Ocena modelu na zbiorze treningowym
train_loss, train_accuracy = model.evaluate(train_generator)
print(f"Strata na zbiorze treningowym: {train_loss}")
print(f"Dokładność na zbiorze treningowym: {train_accuracy}")

# Ocena modelu na zbiorze testowym
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Strata na zbiorze testowym: {test_loss}")
print(f"Dokładność na zbiorze testowym: {test_accuracy}")

25/25 ━━━━━━━━━━━━━━━━━━━━ 59s 2s/step - accuracy: 0.4821 - loss: 1.1197
Strata na zbiorze treningowym: 1.105595588684082
Dokładność na zbiorze treningowym: 0.4967658519744873
7/7 ━━━━━━━━━━━━━━━━━━━━ 10s 1s/step - accuracy: 0.4565 - loss: 1.1191
Strata na zbiorze testowym: 1.108178973197937
Dokładność na zbiorze testowym: 0.47179487347602844


**Proof of Concept - przykład na zdjęciu**


In [16]:
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from PIL import Image

# Słownik z komunikatami dla każdej klasy
class_messages = {
    "PecherzykPrzedowulacyjny": "Zwierzę jest gotowe do owulacji. Zalecana inseminacja w ciągu najbliższych 12 godzin, aby zwiększyć szanse na sukces reprodukcyjny.",
    "PecherzykSredni": "Zwierzę znajduje się w środkowej fazie cyklu. Owulacja jeszcze nie nastąpiła – kontynuuj monitorowanie za 4 dni.",
    "PecherzykDuzy": "Pęcherzyki osiągnęły zaawansowane stadium rozwoju. Zaplanuj inseminację w ciągu najbliższych 24–36 godzin, aby wykorzystać optymalne okno czasowe.",
    "PecherzykMaly": "Pęcherzyki są na wczesnym etapie rozwoju. Zalecane ponowne badanie za 7 dni w celu oceny postępu.",
    "Random": "Nierozpoznano obrazu. Upewnij się czy wysłane zostały poprawne zdjęcia i spróbuj ponownie",
}

# Wczytanie obrazu
img2 = tf.keras.preprocessing.image.load_img(
    '/content/drive/MyDrive/PRZEDOWULACYJNY.jpg',
    target_size=image_size
)
img2 = tf.keras.preprocessing.image.img_to_array(img2) / 255.0
img2 = np.expand_dims(img2, axis=0)

# Predykcja
dataset_prediction = model.predict(img2)
predicted_label = labels[np.argmax(dataset_prediction)]

# Wyświetlanie komunikatu
print(f"Predykcja na podstawie obrazu: {predicted_label}")
if predicted_label in class_messages:
    print("Komunikat:", class_messages[predicted_label])
else:
    print("Komunikat: Predykcja niemożliwa. Upewnij się, że zdjęcie jest poprawne.")

# Opcjonalnie: Wyświetlanie obrazu
img = Image.open('/content/drive/MyDrive/PRZEDOWULACYJNY.jpg')
plt.imshow(img)
plt.axis('off')
plt.show()


1/1 ━━━━━━━━━━━━━━━━━━━━ 6s 6s/step


KeyboardInterrupt: 